# Getting Data from Internet

In [2]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-02-27 19:15:48--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
正在解析主机 raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
正在连接 raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... 已连接。


已发出 HTTP 请求，正在等待回应... 200 OK
长度：1115394 (1.1M) [text/plain]
正在保存至: “input.txt.1”

input.txt.1         100%[===================>]   1.06M  5.79MB/s  用时 0.2s      

2024-02-27 19:15:49 (5.79 MB/s) - 已保存 “input.txt.1” [1115394/1115394])



In [3]:
# read it and inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [4]:
print("length of the dataset in characters: ", len(text))

length of the dataset in characters:  1115394


In [5]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



To see individual character of the whole text

In [6]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


# Encoder and Decoder

## stoi and encoder
stoi 是一个字典，它将字符映射到整数。字典的名称 stoi 是 "string to integer" 的缩写。这个字典是通过字典推导式创建的，字典推导式的输入是 enumerate(chars) 的结果。enumerate(chars) 会为 chars 列表中的每个元素生成一个索引，然后我们将字符作为键，索引作为值，存储在 stoi 字典中。

例如，如果 chars 是 ['a', 'b', 'c']，那么 stoi 就会是 {'a': 0, 'b': 1, 'c': 2}。

encode 是一个函数，它接受一个字符串 s，并返回一个整数列表。这个函数是通过 lambda 表达式定义的，lambda 表达式是一种创建小型匿名函数的方式。encode 函数的工作方式是，对于输入字符串 s 中的每个字符 c，它都会查找 stoi 字典中 c 对应的值（即 stoi[c]），然后将这些值收集到一个列表中。

## itos and decoder
itos 是一个字典，它将整数映射到字符。字典的名称 itos 是 "integer to string" 的缩写。这个字典是通过字典推导式创建的，字典推导式的输入是 enumerate(chars) 的结果。enumerate(chars) 会为 chars 列表中的每个元素生成一个索引，然后我们将索引作为键，字符作为值，存储在 itos 字典中。

例如，如果 chars 是 ['a', 'b', 'c']，那么 itos 就会是 {0: 'a', 1: 'b', 2: 'c'}。

decode 是一个函数，它接受一个整数列表 l，并返回一个字符串。这个函数是通过 lambda 表达式定义的，lambda 表达式是一种创建小型匿名函数的方式。decode 函数的工作方式是，对于输入列表 l 中的每个整数 i，它都会查找 itos 字典中 i 对应的值（即 itos[i]），然后将这些值收集到一个字符串中。

例如，如果 l 是 [0, 1, 2]，那么 decode(l) 就会是 "abc"（假设 itos 是上面的例子中的字典）。


In [7]:
stoi = { ch:i for i,ch in enumerate(chars)}
itos = { i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("Hello NanoGPT"))
print(decode(encode("Hello NanoGPT")))

[20, 43, 50, 50, 53, 1, 26, 39, 52, 53, 19, 28, 32]
Hello NanoGPT


## 使用 Torch 进行包装

In [8]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

# Training Data

In [9]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

在 LLM 训练的时候，一般不会将整个数据库都拿来训练 Transformer，而是会将数据“分块”，一块一块地进行训练

In [10]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

在训练的时候，我们希望 Transformer 能拥有 “预测下一个词“ 的能力。

比如说上述的序列中：
1. 给出一个 ”18“ 的词，Transformer 能够预测下一个词是 ”47“
2. 给出一个序列为 ”18，47“，Transformer 能够预测下一个词是 ”56“
依次类推。

关联信息：[waytoagi.feishu.cn](https://waytoagi.feishu.cn/wiki/LnMOwmKIwihcRakRalcczNgrnjf)

In [11]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


上述的是一个 block 进行训练，真实场景下，我们通常是 “一个批次” 进行训练，即将 “多个 block 合为一个 batch” 进行训练。

以下代码将引入 batch 的维度：

In [12]:
torch.manual_seed(114514) # :)

batch_size = 4  # 定义有多少个batch并行训练     how many independent sequences will we process in parallel?
block_size = 8  # 每一个block的最大长度        what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data

    ix = torch.randint(len(data) - block_size, (batch_size,))   # batch_size 指定生成的长度
    #print('ix: ', str(ix))

    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----------------')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[57, 43, 50, 63,  1, 39, 58,  1],
        [46,  1, 61, 39, 57, 46,  5, 42],
        [58,  1, 54, 56, 43, 54, 39, 56],
        [47, 52, 49,  6,  1, 53, 56,  1]])
targets:
torch.Size([4, 8])
tensor([[43, 50, 63,  1, 39, 58,  1, 51],
        [ 1, 61, 39, 57, 46,  5, 42,  1],
        [ 1, 54, 56, 43, 54, 39, 56, 43],
        [52, 49,  6,  1, 53, 56,  1, 40]])
----------------
when input is [57] the target: 43
when input is [57, 43] the target: 50
when input is [57, 43, 50] the target: 63
when input is [57, 43, 50, 63] the target: 1
when input is [57, 43, 50, 63, 1] the target: 39
when input is [57, 43, 50, 63, 1, 39] the target: 58
when input is [57, 43, 50, 63, 1, 39, 58] the target: 1
when input is [57, 43, 50, 63, 1, 39, 58, 1] the target: 51
when input is [46] the target: 1
when input is [46, 1] the target: 61
when input is [46, 1, 61] the target: 39
when input is [46, 1, 61, 39] the target: 57
when input is [46, 1, 61, 39, 57] the target: 46
when inp

我们输入给 Transformer 训练的是这样的数据，每一行都是一个训练样本：
```py
tensor([[43, 50, 63,  1, 39, 58,  1, 51],
        [ 1, 61, 39, 57, 46,  5, 42,  1],
        [ 1, 54, 56, 43, 54, 39, 56, 43],
        [52, 49,  6,  1, 53, 56,  1, 40]])
```


In [13]:
print(xb) # our input to the transformer

tensor([[57, 43, 50, 63,  1, 39, 58,  1],
        [46,  1, 61, 39, 57, 46,  5, 42],
        [58,  1, 54, 56, 43, 54, 39, 56],
        [47, 52, 49,  6,  1, 53, 56,  1]])


# Bigram Language Model: The Simplest LLM Model

## 知识科普：交叉熵cross_entropy

交叉熵（Cross Entropy）是一种在机器学习和信息论中常用的损失函数，特别是在分类问题中。它用于衡量模型预测的概率分布与真实的概率分布之间的差异。

假设我们有一个真实的概率分布 P 和一个模型预测的概率分布 Q，那么 P 和 Q 之间的交叉熵定义为：`H(P, Q) = - Σ P(x) log(Q(x))`

其中，Σ 表示对所有可能的事件 x 的求和。

在分类问题中，真实的概率分布 P 通常是一个 one-hot 编码的向量，也就是说，对于正确的类别，其概率为 1，对于其他类别，其概率为 0。因此，交叉熵损失函数就变成了对于正确类别的预测概率的负对数似然。

交叉熵损失函数有一个很好的性质，那就是当模型的预测概率分布接近真实的概率分布时，交叉熵的值会很小，反之则会很大。因此，通过最小化交叉熵损失函数，我们可以让模型的预测概率分布尽可能接近真实的概率分布。

> 在以下神经网络中，我们使用 cross_entropy 作为计算 loss 值的方法

In [14]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(114514)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            # https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html
            # 根据pytorch的文档，cross_entropy的入参(minibatch,C,d1,....dk)，第二个维度是C，所以需要对源 logits 进行改造
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 65])
tensor(4.5480, grad_fn=<NllLossBackward0>)

tCK!?.QVDheWEVuNLsj.VwhEgFldy&FueufK.Sl?n3PtXmZvsckrytSgrhgUGl KLepkJ!xSBv,RbXm.g !'JSscvB&NwPDXPJaY


在没有训练的时候，输出的内容都是乱码。

generate 第一个入参是 ‘zero’，在原始文本中表示”空格“，第二个 max_new_tokens 则是让模型推测文本的最大长度

接下来我们将开始做一些简答的训练：

# Traning the Transformer Model

先创建一个 PyTorch 的优化器，用于更新模型的参数以最小化损失函数。

优化器的类型是 AdamW，这是一种常用的优化算法，它结合了 Adam（自适应矩估计）算法的优点和权重衰减（也称为 L2 正则化）的优点。AdamW 优化器在训练深度学习模型时，特别是在处理大规模数据集和复杂模型时，通常可以得到更好的结果。

m.parameters() 是一个生成器，它返回模型 m 的所有可训练参数。这些参数是需要优化的对象。

lr=1e-3 设置了学习率为 0.001。学习率是一个超参数，它决定了模型参数在每次更新时的移动步长。如果学习率太大，可能会导致模型在训练过程中震荡和不稳定；如果学习率太小，可能会导致模型训练速度过慢，甚至可能无法收敛。选择合适的学习率通常需要一些实验和经验。

In [15]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [16]:
batch_size = 32
for steps in range(10000): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


2.524451494216919


In [17]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


c.
NCKIGr ucof im: nhINort isty t
S:
HI:
NCouathe mithidooforikwe thy s IF hasowe la, timid be ele de I's testatemulifouline houliery te---ill my ave acaleailer theavenol, coucok ct wameeereritrera y powe thore corce te t, chewerckestrcy? I malfonose tses
WAnccoreandidindethowifanst hes per ng thes dmy ces:
Laricad spran a ce by lar beg touee yona imyon'd t whowathinge RD mee ws leche,
Ay.
INCUSIndre he l w

O: bestheshf tres balmy alllldire aitheit h aist h t dis?
Ti-yonhe y oworau fove ay.
LOL


在上述的训练方式中，每一个 token 字符之间都没有相互关联，他们不会 'talking to each other'。

接下来，我们将尝试让 token “产生关联”，让训练更有效。

> 注意：bigram 模型的代码都整合为一个 bigram.py 文件了

# Self-Attention 的数学“技巧”

## 0. 前言

我们希望当前的 token 可以与前面的 token 产生“联系”，这样有助于后续 LLM 推理处理。

最简单的做法是在每一个 token 之上，都算一次该 token 之前 tokens 的“平均值”，算作当前 token 的“权重”

In [18]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

这段代码的目标是计算一个新的张量 xbow，其中 xbow[b,t] 是原始张量 x 中前 t+1 个元素的平均值。原始张量 x 的形状是 (B, T, C)，其中 B 是批次大小，T 是序列长度，C 是特征数量。

首先，我们创建了一个全零张量 xbow，其形状与 x 相同。

然后，我们遍历 x 的每一个批次 b 和每一个时间步 t。对于每一个 (b, t) 对，我们取出 x 在批次 b 中前 t+1 个元素，得到一个形状为 (t, C) 的张量 xprev。

接着，我们计算 xprev 在第 0 维（即时间步维度）上的平均值，得到一个形状为 (C,) 的张量。这个操作实际上就是计算了 x 在批次 b 中前 t+1 个元素的特征平均值。

最后，我们将这个平均值赋值给 xbow 在 (b, t) 位置上的元素。这样，xbow[b,t] 就等于 x[b,i] 在 i<=t 时的平均值。

这个过程会对 x 的所有批次和时间步进行，所以最后得到的 xbow 中的每一个元素都是对应的 x 中的元素的前缀平均值。

In [19]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)   x[b,:t+1] 的意思是选取第 b 个批次中的前 t+1 个元素。
        xbow[b,t] = torch.mean(xprev, 0)
xbow


tensor([[[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]],

        [[ 1.3488, -0.1396],
         [ 0.8173,  0.4127],
         [-0.1342,  0.4395],
         [ 0.2711,  0.4774],
         [ 0.2421,  0.0694],
         [ 0.0084,  0.0020],
         [ 0.0712, -0.1128],
         [ 0.2527,  0.2149]],

        [[-0.6631, -0.2513],
         [ 0.1735, -0.0649],
         [ 0.1685,  0.3348],
         [-0.1621,  0.1765],
         [-0.2312, -0.0436],
         [-0.1015, -0.2855],
         [-0.2593, -0.1630],
         [-0.3015, -0.2293]],

        [[ 1.6455, -0.8030],
         [ 1.4985, -0.5395],
         [ 0.4954,  0.3420],
         [ 1.0623, -0.1802],
         [ 1.1401, -0.4462],
         [ 1.0870, -0.4071],
         [ 1.0430, -0.1299],
         [ 1.1138, -0.1641]]])

In [20]:
print(x[0])
print(xbow[0])

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])
tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])


可以看到， x[0] 和 xbow[0] 的第一行都是一样的，不过从第二行开始，每一行都等于之前的序列和的平均值。

以第二行为例：
- 左列：(0.1808 + -0.3596) / 2 = -0.0895
- 右列：(-0.0700 + -0.9151) / 2 = -0.4926

## 1. 使用矩阵乘法优化

上述的做法属于很简单的加法处理，在矩阵中有更快的算法实现

下面是一个示例：
>大二线性代数知识，矩阵乘法（左行✖️右列）

In [21]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)

a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b

print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


上述的矩阵乘法操作就相当于 “每一个 token 都计算之前 tokens 的平均值”，而 a 矩阵就是实现这个算法的“计算矩阵”

矩阵算法讲解：
1. torch.sum(a, 1, keepdim=True) 是计算 a 在第 1 维（即每一行）上的元素之和。keepdim=True 参数的作用是保持输出的维度。也就是说，如果 a 是一个二维张量，那么这个求和操作的结果也是一个二维张量，只不过其第 1 维的长度为 1。
2. 然后，a / torch.sum(a, 1, keepdim=True) 就是将 a 的每一个元素都除以其所在行的元素之和。这个操作是广播进行的，也就是说，每一行的所有元素都会被该行的元素之和所除。
3. 这样，每一行的元素之和就变为了 1，也就是说，每一行都变成了一个概率分布。




将上述算法更换为“矩阵乘法”实现：

In [25]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)

torch.allclose(xbow, xbow2)  # 判断是否相等

True

## 2. 使用 softmax 实现

In [26]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf')) # 此处设置 -inf 的实际意义是：这说明当前 token 无法与未来的 tokens 进行关联
wei = F.softmax(wei, dim=-1)

xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True

这段代码的目标是使用 Softmax 函数来计算一个新的张量 xbow3，并检查它是否与已有的张量 xbow 相近。

首先，我们创建了一个 T x T 的下三角矩阵 tril，其中所有元素都是 1。我们使用 torch.tril 函数来创建这个下三角矩阵。

然后，我们创建了一个全零的 T x T 矩阵 wei。

接着，我们使用 masked_fill 函数将 wei 中对应 tril 中为 0 的位置的元素都填充为负无穷。这样，wei 中的上三角部分（不包括对角线）都变为了负无穷。

然后，我们对 wei 应用了 Softmax 函数，得到了一个新的 wei。在这个操作中，每一行都被转换为了一个概率分布，也就是说，每一行的元素之和为 1。由于 wei 中的上三角部分都是负无穷，所以在应用 Softmax 函数后，这部分的元素都变为了 0。

接着，我们通过矩阵乘法 wei @ x 计算了 wei 和 x 的加权聚合结果 xbow3。在这个操作中，wei 中的每一行都会与 x 进行点积运算，得到 xbow3 中的对应元素。因为 wei 中的每一行都是一个概率分布，所以这个操作实际上就是对 x 进行了加权平均。

最后，我们使用 torch.allclose 函数检查 xbow 和 xbow3 是否相近。如果它们的所有元素都非常接近，那么这个函数就会返回 True。

In [29]:
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [32]:
# Without the softmax function processing
tril = torch.tril(torch.ones(T, T))
wei2 = torch.zeros((T,T))
wei2 = wei2.masked_fill(tril == 0, float('-inf'))
#wei = F.softmax(wei, dim=-1)
wei2

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

此处上三角的 -inf 的实际意义在于：当前的 token 无法与未来的 tokens 进行关联，所以是 -inf

而下三角的概率分布，则说明当前 token 与前面的 tokens 有多大的关联。

基于这个一个特点，我们将构建 Self-Attention 注意力机制！！！

In [33]:
# 最初的结果
print(wei)
print(xbow[0])
print(xbow3[0])

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])
tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])
tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.09

## 3. 基于 self-attention 实现！

In [36]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])